In [31]:
import pickle
import numpy as np
import pandas as pd

In [32]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [42]:
test[['x','y']].iloc[0]

x    [409.0, 419.0, 422.0, 423.0, 422.0, 418.0, 409...
y    [136.0, 148.0, 159.0, 171.0, 187.0, 205.0, 237...
Name: 415, dtype: object

In [43]:
predicted_test = test.copy()

feature = ['x','y']

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

log_likelihoods = pd.DataFrame()
log_likelihoods['predicted_class'] = classes
log_like_max = []
top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
is_predicted = []

for i in range(len(test)):

    true_label = test['label'].iloc[i]
    list_log_likelihoods = []
    for model in models:
        
        angle = test[feature].iloc[i]

        X = [[one] for one in angle]
        list_log_likelihoods.append(model.score(X))

        # list_log_likelihoods.append(model.predict(angle))
        # pass

    log_likelihoods['log_likelihood'] = list_log_likelihoods
    log_likelihoods['true_class'] = true_label

    # Compute top 1 prediction
    max_log_likelihood = log_likelihoods.loc[log_likelihoods['log_likelihood'].idxmax()]
    log_like_max.append(max_log_likelihood['log_likelihood'])
    predicted_class.append(max_log_likelihood['predicted_class'])

    if max_log_likelihood['predicted_class'] == max_log_likelihood['true_class']:
        is_predicted.append(True)
    else:
        is_predicted.append(False)

    # Compute top 3 predictions
    top_3_loglikelihood.append(sorted(list_log_likelihoods, reverse=True)[:3])
    top1, top2, top3 = sorted(list_log_likelihoods, reverse=True)[:3]
    top_3_list = []
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top1]['predicted_class'].values[0])
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top2]['predicted_class'].values[0])
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top3]['predicted_class'].values[0])
    top_3_predicted_classes.append(top_3_list)




predicted_test["predicted_class"] = predicted_class
predicted_test["log_likelihood"] = log_like_max
predicted_test["is_predicted"] = is_predicted
predicted_test["top_3_predicted"] = top_3_predicted_classes
predicted_test["top_3_loglikelihood"] = top_3_loglikelihood

predicted_test['log_likelihood']

ValueError: Found array with dim 3. Estimator expected <= 2.

In [35]:
# X = [[one] for one in test['angles'].iloc[0]]
# model.predict_proba(X)

In [36]:
# Calculate top 3 accuracy
sum_top_3 = 0
for idx, row in predicted_test.iterrows():
    for three_likelihood in row["top_3_predicted"]:
        if(row["label"] == three_likelihood):
            sum_top_3 += 1
            pass

accuracy = sum(predicted_test["is_predicted"])/len(predicted_test["is_predicted"])
print("Accuracy: ", accuracy)
accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy:  0.8145161290322581
Accuracy top 3:  0.967741935483871
